# Output of the symbolic tendencies: AUTO example 

In this notebook, we show how to create the symbolic tendencies of the model and use them to perform continuation of solutions as parameters are varied, using the [AUTO-07p](https://github.com/auto-07p/auto-07p) continuation software. Symbolic tendencies here means that it is possible to make any parameter of the model appears in the tendencies equations.

The present notebook will create the Fortran tendencies equations code and insert it into a given AUTO template.
This can then be used inside the notebook directly.

More details about the model used in this notebook can be found in the articles:
* Hamilton, O., Demaeyer, J., Vannitsem, S., & Crucifix, M. (2025). *Using Unstable Periodic Orbits to Understand Blocking Behaviour in a Low Order Land-Atmosphere Model*. Submitted to Chaos. [preprint](https://doi.org/10.48550/arXiv.2503.02808)
* Xavier, A. K., Demaeyer, J., & Vannitsem, S. (2024). *Variability and predictability of a reduced-order land–atmosphere coupled model.* Earth System Dynamics, **15**(4), 893-912. [doi:10.5194/esd-15-893-2024](https://doi.org/10.5194/esd-15-893-2024)

or in the documentation. In particular, Hamilton et. al. (2025) used the symbolic tendencies here provided to compute periodic orbits, along with an automatic layer for AUTO called [auto-AUTO](https://github.com/Climdyn/auto-AUTO).

> **To run this notebook, you need AUTO properly installed and configured !**
>
> **In general, it means that typing** `auto` **in a terminal starts the AUTO Python interface.**

## Modules import

First, setting the path and loading of some modules

In [ ]:
import sys, os
sys.path.extend([os.path.abspath('../../')])

In [ ]:
import glob

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Importing the model's modules

In [ ]:
from qgs.params.params import QgParams

In [ ]:
from qgs.functions.symbolic_tendencies import create_symbolic_tendencies

## Systems definition

Setting some model parameters

In [ ]:
model_parameters = QgParams({'phi0_npi': np.deg2rad(50.)/np.pi, 'n':1.3 }, dynamic_T=False)

and defining the spectral modes used by the model (they must be *symbolic*)

In [ ]:
# Mode truncation at the wavenumber 2 in both x and y spatial coordinate for the atmosphere
model_parameters.set_atmospheric_channel_fourier_modes(2, 2, mode="symbolic")
# Same modes for the ground temperature modes
model_parameters.set_ground_channel_fourier_modes(2, 2, mode="symbolic")

Completing the model parameters

In [ ]:
# Changing (increasing) the orography depth
model_parameters.ground_params.set_orography(0.2, 1)
# Setting the parameters of the heat transfer from the soil
model_parameters.gotemperature_params.set_params({'gamma': 1.6e7, 'T0': 300})
model_parameters.atemperature_params.set_params({ 'hlambda':10, 'T0': 290})
# Setting atmospheric parameters
model_parameters.atmospheric_params.set_params({'sigma': 0.2, 'kd': 0.085, 'kdp': 0.02})

# Setting insolation 
model_parameters.gotemperature_params.set_params({})

In [ ]:
C_g = 300
model_parameters.atemperature_params.set_insolation(0.4*C_g , 0)

model_parameters.gotemperature_params.set_insolation(C_g , 0)

In [ ]:
# Printing the model's parameters
model_parameters.print_params()

## Creating AUTO files

Calculating the tendencies in Fortran for AUTO as a function of the parameters $C_{{\rm o},0}$, $C_{{\rm a},0}$, $k_d$ and $k'_d$:

In [ ]:
funcs, = create_symbolic_tendencies(model_parameters, continuation_variables=[model_parameters.gotemperature_params.C[0], model_parameters.atemperature_params.C[0], model_parameters.atmospheric_params.kd, model_parameters.atmospheric_params.kdp], language='auto')

Let's inspect the output. First the AUTO `.f90` file:

In [ ]:
print(funcs[0])

and then AUTO `c.` configuration file:

In [ ]:
print(funcs[1])

We can now use both to write the AUTO files.
First we will modify the tendencies to force $C_{{\rm a}, 1} = 0.4 C_{{\rm g}, 1}$ , which is a standard assumption for these models, and reduces the number of parameters to define :

In [ ]:
# splitting all the lines of the .f90 file
auto_eq_lines = funcs[0].split('\n')

In [ ]:
# forcing the change
for i, line in enumerate(auto_eq_lines):
    if 'C_a1 = PAR(2)' in line:
        auto_eq_lines[i] = '\tC_a1 = 0.4*C_go1'
        break

In [ ]:
# gathering all the lines again in a single string
auto_eq = '\n'.join(auto_eq_lines)

Taking care of the config file, changing some default settings:

In [ ]:
# splitting all the lines of the c. file
auto_config_lines = funcs[1].split('\n')

In [ ]:
# introducing some user defined points for AUTO
for i, line in enumerate(auto_config_lines):
    if 'UZR' in line:
        auto_config_lines[i] = "UZR = {'C_go1': " + str(list(np.arange(50.,375.,50.)))+"}"
        break

In [ ]:
# imposing that C_go1 is between 0. and 400. as stopping condition for AUTO
for i, line in enumerate(auto_config_lines):
    if 'UZSTOP' in line:
        auto_config_lines[i] = "UZSTOP = {'C_go1': [0.,400.]}"
        break

In [ ]:
# gathering all the lines again in a single string
auto_config = '\n'.join(auto_config_lines)

and writing to files:

In [ ]:
model_name = 'qgs_land-atmosphere_auto'
with open(f'{model_name}.f90', 'w') as ff:
    ff.write(auto_eq)
    
with open(f'c.{model_name}', 'w') as ff:
    ff.write(auto_config)

## Defining some plotting functions

to help us investigate the results later:

In [ ]:
def plot_branches(filename, variables=(0,1), ax=None, figsize=(10, 8), markersize=12., plot_kwargs=None, marker_kwargs=None, branch_indices='all', excluded_labels=('UZ', 'EP', 'No Label'), variables_name=None):
    
    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = fig.gca()
        
    if plot_kwargs is None:
        plot_kwargs = dict()
        
    if marker_kwargs is None:
        marker_kwargs = dict()
    
    pb_obj = parseB.parseB()
    fb = open(filename, 'r')
    pb_obj.read(fb)
    
    keys = list(pb_obj.branches[0].keys())
    
    if variables[0] in keys:
        var1 = variables[0]
    else:
        try:
            var1 = keys[variables[0]]
        except:
            var1 = keys[0]

    if variables[1] in keys:
        var2 = variables[1]
    else:
        try:
            var2 = keys[variables[1]]
        except:
            var2 = keys[1]

    if branch_indices == 'all':
        branch_indices = range(len(pb_obj.branches))

    branch_num = list()
    for i in branch_indices:
        branch_dict = pb_obj.branches[i].todict()
        branch_num.append(pb_obj.branches[i]['BR'])

        labels = list()
        for j, coords in enumerate(zip(branch_dict[var1], branch_dict[var2])):
            lab = pb_obj.branches[i].labels[j]
            if not lab:
                pass
            else:
                labels.append((coords, list(lab.keys())[0]))

        ax.plot(branch_dict[var1], branch_dict[var2], **plot_kwargs)
        if excluded_labels != 'all':
            for label in labels:
                coords = label[0]
                lab = label[1]
                if lab not in excluded_labels:
                    ax.text(coords[0], coords[1], r'${\bf '+ lab + r'}$', fontdict={'family':'sans-serif','size':markersize},va='center', ha='center', **marker_kwargs, clip_on=True)
    
    fb.close()
    if variables_name is None:
        ax.set_xlabel(var1)
        ax.set_ylabel(var2)
    else:
        if isinstance(variables_name, dict):
            ax.set_xlabel(variables_name[var1])
            ax.set_ylabel(variables_name[var2])
        else:
            ax.set_xlabel(variables_name[0])
            ax.set_ylabel(variables_name[1])
    return ax, branch_num

In [ ]:
def plot_branches3d(filename, variables=(0,1,3), ax=None, figsize=(10, 8), markersize=12., plot_kwargs=None, marker_kwargs=None, branch_indices='all', excluded_labels=('UZ', 'EP', 'No Label'), variables_name=None):
    
    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = plt.subplot(projection='3d')
        
    if plot_kwargs is None:
        plot_kwargs = dict()
        
    if marker_kwargs is None:
        marker_kwargs = dict()
    
    pb_obj = parseB.parseB()
    fb = open(filename, 'r')
    pb_obj.read(fb)
    
    keys = list(pb_obj.branches[0].keys())
    
    if variables[0] in keys:
        var1 = variables[0]
    else:
        try:
            var1 = keys[variables[0]]
        except:
            var1 = keys[0]

    if variables[1] in keys:
        var2 = variables[1]
    else:
        try:
            var2 = keys[variables[1]]
        except:
            var2 = keys[1]
            
    if variables[2] in keys:
        var3 = variables[2]
    else:
        try:
            var3 = keys[variables[2]]
        except:
            var3 = keys[2]


    if branch_indices == 'all':
        branch_indices = range(len(pb_obj.branches))

    branch_num = list()
    for i in branch_indices:
        branch_dict = pb_obj.branches[i].todict()
        branch_num.append(pb_obj.branches[i]['BR'])

        labels = list()
        for j, coords in enumerate(zip(branch_dict[var1], branch_dict[var2], branch_dict[var3])):
            lab = pb_obj.branches[i].labels[j]
            if not lab:
                pass
            else:
                labels.append((coords, list(lab.keys())[0]))

        ax.plot(branch_dict[var1], branch_dict[var2], branch_dict[var3], **plot_kwargs)
        if excluded_labels != 'all':
            for label in labels:
                coords = label[0]
                lab = label[1]
                if lab not in excluded_labels:
                    ax.text(coords[0], coords[1], coords[2], r'${\bf '+ lab + r'}$', fontdict={'family':'sans-serif','size':markersize},va='center', ha='center', **marker_kwargs, clip_on=True)
    
    fb.close()
    if variables_name is None:
        ax.set_xlabel(var1)
        ax.set_ylabel(var2)
        ax.set_zlabel(var3)
    else:
        if isinstance(variables_name, dict):
            ax.set_xlabel(variables_name[var1])
            ax.set_ylabel(variables_name[var2])
            ax.set_zlabel(variables_name[var3])
        else:
            ax.set_xlabel(variables_name[0])
            ax.set_ylabel(variables_name[1])
            ax.set_zlabel(variables_name[2])
    return ax, branch_num

In [ ]:
def plot_branch_vs_others(branch_num, figsize=(10, 16), excluded_labels=('UZ', 'EP', 'No Label')):
    
    fig = plt.figure(figsize=figsize)
    ax = plt.subplot(2,1,1)
    ax3 = plt.subplot(2,1,2, projection='3d')
    
    
    
    fp = glob.glob('./b.fp*')
    fp = [item for item in fp if '~' not in os.path.basename(item)]
    fp = [item for item in fp if '_' not in os.path.basename(item)]
    
    for i in range(len(fp)-1,-1,-1):
    
        try:
            num = int(fp[i][-2:])
        except:
            num = int(fp[i][-1])
            
        if num == branch_num:
            plot_branches(fp[i], ax=ax, plot_kwargs={'color': 'tab:blue', 'zorder': 10.}, variables=(0, 1), variables_name=(r'$C_{\rm o}$', r'$L_2$ norm'), excluded_labels=excluded_labels)
            plot_branches3d(fp[i], ax=ax3, plot_kwargs={'color': 'tab:blue', 'zorder': 10.}, variables=(3, 0, 1), variables_name=(r'$\psi_{{\rm a}, 2}$', r'$C_{\rm o}$', r'$L_2$ norm'), excluded_labels=excluded_labels)
        else:
            plot_branches(fp[i], ax=ax, plot_kwargs={'color': 'tab:orange'}, variables=(0, 1), variables_name=(r'$C_{\rm o}$', r'$L_2$ norm'), excluded_labels="all")
            plot_branches3d(fp[i], ax=ax3, plot_kwargs={'color': 'tab:orange'}, variables=(3, 0, 1), variables_name=(r'$\psi_{{\rm a}, 2}$', r'$C_{\rm o}$', r'$L_2$ norm'), excluded_labels="all")

            

## AUTO analysis

Initializing AUTO

In [ ]:
# Finding where AUTO is installed
auto_directory = os.environ['AUTO_DIR']

In [ ]:
# Adding it to the path
sys.path.append(auto_directory + '/python/auto')
sys.path.append(auto_directory + '/python')

In [ ]:
# Loading the needed AUTO Python interfaces
import AUTOCommands as ac
import AUTOclui as acl
import interactiveBindings as ib
import runAUTO as ra
import parseB, parseC, parseD, parseS, parseBandS

Loading the model

In [ ]:
print("Loading model "+model_name)

Starting a runner

In [ ]:
runner = ra.runAUTO()
ac.load(model_name, runner=runner)

Finding the first branch of fixed point

In [ ]:
U_dic = {i+1: 0. for i in range(model_parameters.ndim)}
x = ac.run(model_name, U=U_dic, ICP=['C_go1'], PAR={3: model_parameters.atmospheric_params.kd, 4: model_parameters.atmospheric_params.kdp}, runner=runner)
ac.save(x,'fp1')

In [ ]:
plot_branch_vs_others(1)

## Computing the periodic orbits (POs) out of the fixed point

Loading the branch and printing the summary

In [ ]:
r = ac.loadbd('fp1')
print(r.summary())

Listing the Hopf bifurcation points

In [ ]:
solutions_list = list()
ps_obj = parseS.parseS('./s.fp1')
pc_full_obj = parseC.parseC('c.'+model_name)
for i in range(len(ps_obj)):
    s = ps_obj[i].load(constants=pc_full_obj)
    if s['TY'] == 'HB':
        solutions_list.append(s)
    

# reversing to get it in Co increasing order
solutions_list = solutions_list[::-1]

In [ ]:
solutions_list

### Computing and plotting the second Hopf bifurcation

In [ ]:
s = solutions_list[1]
rh=ac.run(s,ICP=['C_go1', 'T'], IPS=2, NTST=400, runner=runner)
ac.save(rh, 'fp1_hp1')

In [ ]:
ax, _ = plot_branches('./b.fp1_hp1', variables=(0, 3))
plot_branches('./b.fp1', ax=ax, variables=(0, 3))

Other fixed point and periodic orbit branches can of course be computed in the same way.